In [37]:


import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import re
import warnings

warnings.filterwarnings('ignore')




In [ ]:
# 1. CARGA DE DATOS Y LIMPIEZA INICIAL

# Cargamos los datos 
try:
    train = pd.read_csv("train.csv")
    test_df = pd.read_csv("test.csv")
    print(f"✅ Datos cargados: {train.shape[0]} filas de entrenamiento.")
except FileNotFoundError:
    print("❌ Error: No se encuentran 'train.csv' o 'test.csv'. Verifica la ruta.")
    raise

# Imputación de Valores Nulos (Fundamental antes de crear features)
# Numéricos -> Mediana | Categóricos -> Moda
def imputar_nulos(df):
    for col in df.columns:
        if df[col].isnull().any():
            if df[col].dtype == 'object':
                df[col] = df[col].fillna(df[col].mode()[0])
            else:
                df[col] = df[col].fillna(df[col].median())
    return df

train = imputar_nulos(train)
test_df = imputar_nulos(test_df)
print("✅ Valores nulos imputados correctamente.")

✅ Datos cargados: 692500 filas de entrenamiento.
✅ Valores nulos imputados correctamente.


In [ ]:

# 2. FEATURE ENGINEERING agregando Std y Rangos


cols_indicadores = [col for col in train.columns if 'INDICADOR_' in col]

def crear_features_inteligentes(df):
    # Promedio (Señal base)
    df['INDICADOR_PROMEDIO'] = df[cols_indicadores].mean(axis=1)
    
    # Desviación Estándar (Detecta estudiantes inestables vs constantes)
    df['INDICADOR_STD'] = df[cols_indicadores].std(axis=1)
    
    # Min y Max (Techo y suelo de rendimiento)
    df['INDICADOR_MIN'] = df[cols_indicadores].min(axis=1)
    df['INDICADOR_MAX'] = df[cols_indicadores].max(axis=1)
    
    # Rango (Amplitud de habilidades)
    df['INDICADOR_RANGO'] = df['INDICADOR_MAX'] - df['INDICADOR_MIN']
    
    return df

train = crear_features_inteligentes(train)
test_df = crear_features_inteligentes(test_df)
print("✅ Ingeniería de características aplicada (Promedio, Std, Rango).")

✅ Ingeniería de características aplicada (Promedio, Std, Rango).


In [ ]:

# 3. PREPROCESAMIENTO 
# Guardamos IDs para el final
train_ids = train['ID']
test_ids = test_df['ID']
target_col = 'RENDIMIENTO_GLOBAL'

# Codificar Variable Objetivo (y)
le = LabelEncoder()
y_encoded = le.fit_transform(train[target_col])

# Separar variables predictoras
X = train.drop(['ID', target_col], axis=1)
X_test_real = test_df.drop(['ID'], axis=1)

# Concatenar para One-Hot Encoding (Garantiza mismas columnas siempre)
X_combined = pd.concat([X, X_test_real], keys=['train', 'test'])
X_combined = pd.get_dummies(X_combined, drop_first=True)

# Limpieza de nombres de columnas para XGBoost (Elimina caracteres especiales)
X_combined.columns = [re.sub(r'[<>()\[\]{},.:;\'\"-/]', '_', col) for col in X_combined.columns]

# Separar nuevamente
X_train_final = X_combined.loc['train'].reset_index(drop=True)
X_test_final = X_combined.loc['test'].reset_index(drop=True)

print(f"✅ Dimensiones finales alineadas -> Train: {X_train_final.shape}, Test: {X_test_final.shape}")

✅ Dimensiones finales alineadas -> Train: (692500, 1043), Test: (296786, 1043)


In [ ]:

# 4. CONFIGURACIÓN Y ENTRENAMIENTO 

X_t, X_val, y_t, y_val = train_test_split(
    X_train_final, 
    y_encoded, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_encoded
)

# Parámetros optimizados
xgb_params = {
    'objective': 'multi:softmax',
    'num_class': len(le.classes_),
    'n_estimators': 3000,
    'learning_rate': 0.02,
    'max_depth': 6,
    'subsample': 0.75,
    'colsample_bytree': 0.65,
    'min_child_weight': 1,
    'gamma': 0.1,
    'eval_metric': 'mlogloss',
    'n_jobs': -1,
    'random_state': 42,
    'tree_method': 'hist'
}

print("⚙️ Entrenando modelo XGBoost Super-Tuned...")

model_xgb = xgb.XGBClassifier(**xgb_params)


# CONVERTIMOS EXPLICITAMENTE LAS MATRICES X A ARRAYS DE NUMPY USANDO .values
model_xgb.fit(
    X_t.values, 
    y_t,
    eval_set=[(X_t.values, y_t), (X_val.values, y_val)],
    verbose=200
)

print("🎉 ¡Modelo entrenado exitosamente!")

⚙️ Entrenando modelo XGBoost Super-Tuned (esto puede tardar unos minutos)...
[0]	validation_0-mlogloss:1.38376	validation_1-mlogloss:1.38381
[200]	validation_0-mlogloss:1.25651	validation_1-mlogloss:1.26131
[400]	validation_0-mlogloss:1.23329	validation_1-mlogloss:1.24134
[600]	validation_0-mlogloss:1.22014	validation_1-mlogloss:1.23095
[800]	validation_0-mlogloss:1.21092	validation_1-mlogloss:1.22431
[1000]	validation_0-mlogloss:1.20382	validation_1-mlogloss:1.21953
[1200]	validation_0-mlogloss:1.19802	validation_1-mlogloss:1.21596
[1400]	validation_0-mlogloss:1.19301	validation_1-mlogloss:1.21309
[1600]	validation_0-mlogloss:1.18870	validation_1-mlogloss:1.21079
[1800]	validation_0-mlogloss:1.18474	validation_1-mlogloss:1.20883
[2000]	validation_0-mlogloss:1.18118	validation_1-mlogloss:1.20714
[2200]	validation_0-mlogloss:1.17798	validation_1-mlogloss:1.20575
[2400]	validation_0-mlogloss:1.17493	validation_1-mlogloss:1.20449
[2600]	validation_0-mlogloss:1.17209	validation_1-mlogloss:

In [ ]:

# 5. GENERACIÓN DE RESULTADOS Y ARCHIVO DE ENVÍO

print("📝 Generando predicciones...")

# Predecir

preds_numeric = model_xgb.predict(X_test_final.values) 
# Decodificar 
preds_text = le.inverse_transform(preds_numeric)

# Crear DataFrame final
submission_pro = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': preds_text
})

# Guardar
nombre_archivo = 'submission_xgb_features_avanzadas_v2.csv'
submission_pro.to_csv(nombre_archivo, index=False)

print(f"✅ Archivo generado: {nombre_archivo}")
print("Muestra de las primeras 5 filas:")
print(submission_pro.head())

📝 Generando predicciones...
✅ Archivo generado: submission_xgb_features_avanzadas_v2.csv
Muestra de las primeras 5 filas:
       ID RENDIMIENTO_GLOBAL
0  550236               bajo
1   98545         medio-alto
2  499179               alto
3  782980               bajo
4  785185               bajo


In [ ]:
# --- IMPORTS ADICIONALES ---
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression # Nuestro meta-modelo
import lightgbm as lgb
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')


In [ ]:

# 6. ENTRENAMIENTO DE MODELOS BASE CON STACKING (LightGBM y XGBoost)


NFOLDS = 5 # Número de pliegues (splits)
SEED = 42
skf = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)

# Inicializar arrays para almacenar predicciones fuera de pliegue (OOF)
oof_preds_xgb = np.zeros((X_train_final.shape[0], len(le.classes_)))
oof_preds_lgbm = np.zeros((X_train_final.shape[0], len(le.classes_)))

# Inicializar arrays para almacenar predicciones de prueba promediadas
test_preds_xgb = np.zeros((X_test_final.shape[0], len(le.classes_)))
test_preds_lgbm = np.zeros((X_test_final.shape[0], len(le.classes_)))

# --- Parámetros de XGBoost ---
xgb_params = {
    'objective': 'multi:softprob', 
    'num_class': len(le.classes_),
    'n_estimators': 1000,
    'learning_rate': 0.02,
    'max_depth': 6,
    'subsample': 0.75,
    'colsample_bytree': 0.65,
    'eval_metric': 'mlogloss',
    'n_jobs': -1,
    'random_state': SEED,
    'tree_method': 'hist'
}

# --- Parámetros de LightGBM 
lgbm_params = {
    'objective': 'multiclass',
    'num_class': len(le.classes_),
    'n_estimators': 1000,
    'learning_rate': 0.03,
    'num_leaves': 35,
    'reg_lambda': 0.1,
    'random_state': SEED,
    'n_jobs': -1,
    'verbose': -1
}

# ----------------- INICIO DEL K-FOLD -----------------
print(f"⚙️ Iniciando entrenamiento con Stacking Ensemble ({NFOLDS} pliegues)...")

for fold, (train_index, val_index) in enumerate(skf.split(X_train_final, y_encoded)):
    print(f"\n--- Pliegue {fold+1}/{NFOLDS} ---")
    
    # Preparación de datos para el pliegue
    X_train, X_val = X_train_final.iloc[train_index], X_train_final.iloc[val_index]
    y_train, y_val = y_encoded[train_index], y_encoded[val_index]
    
    # 1. Modelo XGBoost Base
    model_xgb = xgb.XGBClassifier(**xgb_params)
    model_xgb.fit(
        X_train.values, y_train,
        eval_set=[(X_val.values, y_val)],
        verbose=False,
    )
    # Almacenar predicciones fuera de pliegue (OOF) y de prueba
    oof_preds_xgb[val_index] = model_xgb.predict_proba(X_val.values)
    test_preds_xgb += model_xgb.predict_proba(X_test_final.values) / NFOLDS

    # 2. Modelo LightGBM 
    model_lgbm = lgb.LGBMClassifier(**lgbm_params)
    model_lgbm.fit(
        X_train.values, y_train,
        eval_set=[(X_val.values, y_val)],
        eval_metric='multi_logloss',
        callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)]
    )
    # Almacenar predicciones fuera de pliegue (OOF) y de prueba
    oof_preds_lgbm[val_index] = model_lgbm.predict_proba(X_val.values)
    test_preds_lgbm += model_lgbm.predict_proba(X_test_final.values) / NFOLDS

print("✅ Entrenamiento de Modelos Base completado.")

⚙️ Iniciando entrenamiento con Stacking Ensemble (5 pliegues)...

--- Pliegue 1/5 ---

--- Pliegue 2/5 ---

--- Pliegue 3/5 ---

--- Pliegue 4/5 ---

--- Pliegue 5/5 ---
✅ Entrenamiento de Modelos Base completado.


In [ ]:

# 5. ENTRENAMIENTO DEL META-MODELO (STACKING)


# Crear el conjunto de datos de Nivel 2 (Meta-Features)
# Las nuevas features son las probabilidades de cada clase (4 clases) de cada modelo (2 modelos)
X_meta = np.hstack((oof_preds_xgb, oof_preds_lgbm))

# Crear el conjunto de datos de prueba de Nivel 2
X_meta_test = np.hstack((test_preds_xgb, test_preds_lgbm))

# Meta-Modelo: Regresión Logística 
print("⚙️ Entrenando Meta-Modelo (Logistic Regression)...")
meta_model = LogisticRegression(
    solver='lbfgs',
    multi_class='multinomial',
    C=0.1, 
    random_state=SEED,
    n_jobs=-1
)

# Entrenar el Meta-Modelo
meta_model.fit(X_meta, y_encoded)
print("✅ Meta-Modelo entrenado.")

⚙️ Entrenando Meta-Modelo (Logistic Regression)...
✅ Meta-Modelo entrenado.


In [20]:

# 6. PREDICCIÓN FINAL Y SUBMISIÓN


# Predicción final del Meta-Modelo
final_probas = meta_model.predict_proba(X_meta_test)

# Obtener la clase con la probabilidad más alta
final_predictions_numeric = np.argmax(final_probas, axis=1)

# Decodificar
final_predictions_text = le.inverse_transform(final_predictions_numeric)

# Crear y guardar el archivo de envío
submission_ensemble_df = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': final_predictions_text
})

nombre_archivo_final = 'submission_STACKING_FINAL.csv'
submission_ensemble_df.to_csv(nombre_archivo_final, index=False)

print(f"\n=========================================================")
print(f"ÉXITO: Stacking Ensemble completado. Archivo generado: {nombre_archivo_final}")
print(submission_ensemble_df.head())


ÉXITO: Stacking Ensemble completado. Archivo generado: submission_STACKING_FINAL.csv
       ID RENDIMIENTO_GLOBAL
0  550236               bajo
1   98545         medio-alto
2  499179               alto
3  782980               bajo
4  785185               bajo
